In [58]:
import tweepy
import csv
import pandas as pd
import numpy as np
import string
import re
import nltk
import Sastrawi
import ast
import os
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

from tweepy import OAuthHandler
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from wordcloud import WordCloud

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn import metrics, preprocessing
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection

In [59]:
import ipywidgets as widgets
from ipywidgets import Output, Button
from IPython.display import HTML, display, clear_output

In [60]:
display(HTML('<h1><center>Sentiment Analysis on Shopefood</h1></center>'))

In [61]:
api_key = "qubRP6d5B5eCD3RHrgoMRkDjH"
api_secret_key = "AJlk1MiHfXqnhByWtCf1Mbi2T9WGW08hoLkkDIbks4ztUpFx99"
access_token ="3180202308-DmrgiSoxUUoSRevLLPldgCE57M0eD117K0mhFG5"
access_token_secret = "fuY5Hkk838OnmGOZUb16KLkSDw3JZjflxXQ1NHP6pP9Jr"

In [16]:
widgets.IntSlider()
w = widgets.IntSlider()
uploader = widgets.FileUpload(
    accept='*.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

display(uploader)

FileUpload(value={}, accept='*.csv', description='Upload')

In [21]:
input_file = list(uploader.value.values())[0]
content = input_file['content']
content = io.StringIO(content.decode('utf-8'))
df = pd.read_csv(content)
display(df)

,Unnamed: 0,Tweets,Case_Folding,Clean_Data,Tweet_Normal,Tanpa_StopWords,Stemming,Bobot,Label,Sentiment
0,0,shopeefood ini ga bisa ngasih tip buat driver ...,shopeefood ini ga bisa ngasih tip buat driver ...,shopeefood ini ga bisa ngasih tip buat driver ya,shopeefood ini tidak bisa memberi tip buat dri...,shopeefood tip driver,"['shopeefood', 'tip', 'driver']",4,positif,2.0
1,1,jangan ujan woyy shopeefood gua belom sampe 😭,jangan ujan woyy shopeefood gua belom sampe 😭,jangan ujan woyy shopeefood gua belom sampe,jangan hujan dong shopeefood saya belum sampai,hujan shopeefood,"['hujan', 'shopeefood']",1,positif,2.0
2,2,Gw masih kepikiran that random cheesecake I fo...,gw masih kepikiran that random cheesecake i fo...,gw masih kepikiran that random cheesecake i fo...,saya masih kepikiran that random cheesecake i ...,kepikiran random cheesecake found shopeefood,"['pikir', 'random', 'cheesecake', 'found', 'sh...",-3,negatif,0.0
3,3,@urlovelymeow Coba cek di shopeefood,@urlovelymeow coba cek di shopeefood,coba cek di shopeefood,coba cek di shopeefood,coba cek shopeefood,"['coba', 'cek', 'shopeefood']",-2,negatif,0.0
4,4,@wannayuuu @bdngfess Sok kirim aja kalo mau mh...,@wannayuuu @bdngfess sok kirim aja kalo mau mh...,sok kirim aja kalo mau mh seriusan ini tersera...,sok kirim aja kalau mau mah seriusan ini terse...,sok kirim mah seriusan terserah maunya grabfoo...,"['sok', 'kirim', 'mah', 'serius', 'serah', 'ma...",4,positif,2.0
...,...,...,...,...,...,...,...,...,...,...
5503,5503,Makan Gratis Bareng Teman di ShopeeFood https:...,makan gratis bareng teman di shopeefood https:...,makan gratis bareng teman di shopeefood,makan gratis bersama teman di shopeefood,makan gratis teman shopeefood,"['makan', 'gratis', 'teman', 'shopeefood']",5,positif,2.0
5504,5504,Geh! Aku mau daftarin ayahku di shopeefood as ...,geh! aku mau daftarin ayahku di shopeefood as ...,geh aku mau daftarin ayahku di shopeefood as d...,geh saya mau daftarin ayahku di shopeefood as ...,geh daftarin ayahku shopeefood driver daftarny...,"['geh', 'daftarin', 'ayah', 'shopeefood', 'dri...",11,positif,2.0
5505,5505,alhamdulillah di rancaekek udh ada shopeefood,alhamdulillah di rancaekek udh ada shopeefood,alhamdulillah di rancaekek udh ada shopeefood,alhamdulillah di rancaekek sudah ada shopeefood,alhamdulillah rancaekek shopeefood,"['alhamdulillah', 'rancaekek', 'shopeefood']",5,positif,2.0
5506,5506,Makan Gratis Bareng Teman di ShopeeFood https:...,makan gratis bareng teman di shopeefood https:...,makan gratis bareng teman di shopeefood,makan gratis bersama teman di shopeefood,makan gratis teman shopeefood,"['makan', 'gratis', 'teman', 'shopeefood']",5,positif,2.0


In [62]:
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret) 
API = tweepy.API(auth, wait_on_rate_limit = False)

In [67]:
input = widgets.Text()
display (input)
btn = widgets.Button(description='Search')
display (btn)
result = widgets.Text()
display (result)

tweets = tweepy.Cursor(API.search_tweets, q=input, lang = "id").items(50)
users = [[tweet.created_at, tweet.full_text] for tweet in tweets]
df = pd.DataFrame(data=users, columns=['Created_At','Tweets'])

def on_button_clicked(b)
    input.value='result' + df
    
btn.on_click(on_button_clicked)

SyntaxError: invalid syntax (Temp/ipykernel_10228/3703256921.py, line 12)